# 要約 
このJupyterノートブックは、Kaggleの「LLM 20 Questions」コンペティションに参加するためのシンプルなスタートを提供します。主な目的は、言語モデル（LLM）を用いて「20の質問」ゲームをプレイするための基本的なフレームワークを提供することです。

### 問題と取り組んでいる内容
ノートブックは、プレイヤーが有限の質問回数でターゲットワード（動物、植物、物体など）を特定する「20の質問」ゲームを効率的にプレイするために、質問者LLMと回答者LLMを活用するアプローチを取っています。実際に選手が質問を行い、その回答に基づいて推測を行うプロセスを自動化しています。

### 手法とライブラリ
1. **使用ライブラリ**:
   - `torch`: PyTorchライブラリを用いてモデルの構築とGPUでの計算を管理しています。
   - `gemma`: Googleが提供するGemmaモデルを使い、因果推論モデルとしての機能を利用しています。
   - その他のPythonライブラリ（`immutabledict`、`sentencepiece`）がインストールされ、モデルの運用を支援しています。

2. **モデル準備**:
   - GitHubから`gemma_pytorch`リポジトリをクローンし、必要なライブラリを作業ディレクトリに移動して設定しています。
   - 重みファイルパスを設定し、Gemmaモデルの初期化を行います。

3. **エージェントクラス**:
   - `Agents`クラスが定義され、3つの主要なエージェント（質問エージェント、推測エージェント、回答エージェント）が実装されています。これらはモデルに基づいて質問や推測を行い、適切な応答を生成します。
   - 各エージェントは与えられた情報を元に動作し、ゲームの進行に合わせて最適なアクションをとります。

4. **プロンプトエンジニアリング**:
   - 各エージェントは、文脈に応じたプロンプトを生成し、そのプロンプトを使用してLLMに問い合わせ、応答を得る形式をとっています。

このノートブックは、ユーザーが簡単にプロンプトエンジニアリングを開始し、提案された改善点や問題提起を通じてさらなる発展を促すことを目的としています。また、実行に必要なコードに加えて、簡潔に試すためのテスト用の例も含まれています。

---


# 用語概説 
以下は、Jupyter Notebook内で扱われている機械学習・深層学習の関連用語について、初心者がつまずきそうな専門用語の簡単な解説です。

### 専門用語解説

1. **プロンプトエンジニアリング (Prompt Engineering)**  
   これは、AIモデル、特に言語モデルに対して入力の形式（プロンプト）を工夫して、期待される出力を得る技術です。質問の仕方や文の構造を調整することで、モデルの応答の質を高めることが目的となります。

2. **immutabledict**  
   イミュータブル（不変）な辞書構造を持つデータ型です。Pythonの辞書は通常、変更可能ですが、`immutabledict`は一度作成すると内容を変更できないため、ハッシュ可能でより安全に使用できます。

3. **sentencepiece**  
   テキストデータをトークン化するためのライブラリです。生のテキストをサブワードやトークンに分割し、言語モデルの学習に使うためのデータ準備を行います。特に、ニューラルネットワークモデルのトレーニングにおいて便利です。

4. **Gemma (GemmaForCausalLM)**  
   確率的生成モデルである「因果言語モデル」を実装したライブラリやクラスです。特に自然言語生成タスクに使用され、多段階の生成が可能です。ここでは、質問応答のエージェントとして機能します。

5. **データクラス (Dataclass)**  
   Python 3.7以降で導入された、簡単に変更可能なデータ構造を作成するための構文です。パラメータを属性として定義し、イミュータブルなオブジェクトを容易に構築できます。これは、データを簡潔に表現するのに便利です。

6. **ターンタイプ (Turn Type)**  
   ゲームの状態に応じてエージェントの行動を識別するための変数です。「質問する」「推測する」「回答する」の3種類があり、それぞれのエージェントがどの役割を果たすかを決定します。

7. **サンプリング (Sampling)**  
   モデルが生成するテキストの選択肢を制御する方法です。このノートブックでは、`temperature`、`top_p`、`top_k`というパラメータがサンプリングに使われ、生成されるテキストの多様性や創造性を調整します。

8. **コンテキストマネージャ (Context Manager)**  
   リソース（例えばファイルやメモリ）の管理を簡単に行うためのPythonの機能です。ここでは、学習時に指定されたデータ型を設定する際に用いられています。

9. **推測エージェント (Guess Agent)**  
   質問応答ゲームにおける特定の役割で、これまでの質問と回答に基づいてキーワードを推測するエージェントです。他のエージェントとの相互作用を通じて進行します。

10. **エンティティ (Entity)**  
    テキストやデータから特定の情報（名前、地名、組織名など）を抽出するプロセスで、質問応答や情報検索システムで重要な役割を果たします。

初心者がこれらの用語を理解できると、ノートブックの内容やその実装に関連する部分がより明確になるでしょう。また、特に実務経験がない場合、コンペティションに特化した知識を得ることは良い学習機会になります。

---


# シンプルスタートを示すノートブック

優れた「LLM 20質問スターターノートブック」を基にしています。  

しかし、これはプロンプトエンジニアリングを迅速に始めるために、はるかに簡素化されています。  

「LLM 20質問スターターノートブック」ほど高度ではなく、プロンプトはコンペティション用に提供された「LLM 20質問」ノートブックから直接取得されています。  

最後には、あなたのプロンプトをテストするためのセルがあります。  

問題や改善の提案は歓迎します。

In [ ]:
%%bash
# 作業ディレクトリに移動します
cd /kaggle/working

# immutabledictとsentencepieceパッケージをインストールします。
# -qオプションは出力を抑制し、-Uはパッケージをアップグレードします。
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece

# gemma_pytorchリポジトリをGitHubからクローンします。
# 出力を/dev/nullにリダイレクトすることで表示を抑制します。
git clone https://github.com/google/gemma_pytorch.git > /dev/null

# gemmaフォルダを作成します。
mkdir /kaggle/working/submission/lib/gemma/

# gemma_pytorchから中身を移動します。
# 移動させることで、gemmaライブラリを作成したlibフォルダに配置します。
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

**モデルを追加するには「Add Input」を使用します**  
これは環境の右側に見つかります。

In [ ]:
# %%writefile submission/main.py

import os
import sys

# Kaggleエージェントのパスを定義します
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
# Kaggleエージェントのパスが存在するか確認します
if os.path.exists(KAGGLE_AGENT_PATH):
    # 存在する場合、ライブラリのパスを挿入します
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    # 存在しない場合、作業スペース内のライブラリのパスを挿入します
    sys.path.insert(0, "/kaggle/working/submission/lib")

import contextlib
import os
import sys
from dataclasses import dataclass
from typing import Literal, List

import torch
from gemma.config import get_config_for_7b
from gemma.model import GemmaForCausalLM

# 重みファイルのパスを設定します
if os.path.exists(KAGGLE_AGENT_PATH):
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/7b-it-quant/2")
else:
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/7b-it-quant/2"


# オブジェクトの内容を把握するためのデータクラスを設定します
@dataclass
class ObsData:
    answers: List["str"]
    category: str
    keyword: str
    questions: List["str"]
    turn_type: Literal["ask", "guess", "answer"]
    

class Agents:
    def __init__(self):
        self.model = None
        
        # 7Bモデルはかなり大きいので、GPUセッションを要求する方が良いです
        self._device = torch.device("cpu")
        if torch.cuda.is_available():
            self._device = torch.device("cuda:0")
    
    def answer_agent(self, question: str, category: str, keyword: str) -> Literal["yes", "no"]:
        info_prompt = """あなたは20の質問のゲームで非常に正確な回答者です。質問者が推測しようとしているキーワードは[カテゴリの {category} {keyword}]です。"""
        answer_question_prompt = f"""次の質問に「はい」、「いいえ」、またはわからない場合は「多分」で答えてください: {question}"""
    
        prompt = "{}{}".format(
            info_prompt.format(category=category, keyword=keyword),
            answer_question_prompt
        )
        
        return self._call_llm(prompt)

    def ask_agent(self, questions: List[str], answers: List[str]) -> str:
        info_prompt = """あなたは20の質問のゲームをプレイしていて、質問をしながらキーワードを特定しようとしています。それは実在の人、場所、または物か架空のものでしょう。\nここまでの知識:\n{q_a_thread}"""
        questions_prompt = """1つの「はい」または「いいえ」の質問をしてください。"""

        q_a_thread = ""
        for i in range(0, len(answers)):
            q_a_thread = "{}Q: {} A: {}\n".format(
                q_a_thread,
                questions[i],
                answers[i]
            )
    
        prompt = "{}{}".format(
            info_prompt.format(q_a_thread=q_a_thread),
            questions_prompt
        )

        return self._call_llm(prompt)

    
    def guess_agent(self, questions: List[str], answers: List[str]) -> str:
        # 期待される回答は**で囲まれた形式です
        info_prompt = """あなたは20の質問のゲームをプレイしていて、質問をしながらキーワードを特定しようとしています。それは実在の人、場所、または物か架空のものでしょう。\nここまでの知識:\n{q_a_thread}"""
        guess_prompt = """キーワードを推測してください。正確な単語/フレーズのみを返してください。たとえば、キーワードが[パリ]だと思うなら、[私はキーワードがパリだと思います]や[キーワードはパリですか？]とは返さず、単に[パリ]と答えてください。"""

        q_a_thread = ""
        for i in range(0, len(answers)):
            q_a_thread = "{}Q: {} A: {}\n".format(
                q_a_thread,
                questions[i],
                answers[i]
            )
        
        prompt = "{}{}".format(
            info_prompt.format(q_a_thread=q_a_thread),
            guess_prompt
        )

        return f"**{self._call_llm(prompt)}**"
    
    def _call_llm(self, prompt: str):
        self._set_model()
        
        sampler_kwargs = {
            'temperature': 0.01,
            'top_p': 0.1,
            'top_k': 1,
        }
        
        return self.model.generate(
            prompt,
            device=self._device,
            output_len=100,
            **sampler_kwargs,
        )
         
    def _set_model(self):
        if self.model is None:
            print("まだモデルが設定されていないため、設定します")
            model_config = get_config_for_7b()
            model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")
            model_config.quant = True

            # コンテキストマネージャを使用しないとスタックが溢れます
            with self._set_default_tensor_type(model_config.get_dtype()):
                model = GemmaForCausalLM(model_config)
                ckpt_path = os.path.join(WEIGHTS_PATH , f'gemma-7b-it-quant.ckpt')
                model.load_weights(ckpt_path)
                self.model = model.to(self._device).eval()
    
    @contextlib.contextmanager
    def _set_default_tensor_type(self, dtype: torch.dtype):
        """指定されたdtypeをデフォルトのtorch dtypeに設定します。"""
        torch.set_default_dtype(dtype)
        yield
        torch.set_default_dtype(torch.float)

# エントリポイント。このため、名前とパラメータは事前に設定されています
def agent_fn(obs, cfg) -> str:
    obs_data = ObsData(
        turn_type=obs.turnType,
        questions=obs.questions,
        answers=obs.answers,
        keyword=obs.keyword,
        category=obs.category
    )
    
    if obs_data.turn_type == "ask":
        response = agents.ask_agent(questions=obs.questions, answers=obs.answers)
    if obs_data.turn_type == "guess":
        response = agents.guess_agent(questions=obs.questions, answers=obs.answers)
    if obs_data.turn_type == "answer":
        response = agents.answer_agent(question=obs.questions[-1], category=obs.category, keyword=obs.keyword)
    
    if response is None or len(response) <= 1:
        return "yes"
    
    return response

# モデルが1回だけ設定されるようにエージェントクラスをインスタンス化します
agents = Agents()

In [ ]:
# 手動で回答エージェントを実行します
@dataclass
class ObsDataIn(ObsData):
    turnType: str
    
obs_data = ObsDataIn(
        turn_type="",
        turnType="answer",  # ターンタイプを「回答」に設定します
        questions=["Is it a place"],  # 質問のリスト
        answers=[],  # まだ回答は存在しないため空のリスト
        keyword="Antartica",  # 推測するキーワード
        category="Place"  # カテゴリーは「場所」
    )

# agent_fnを呼び出して反応を印刷します
print(agent_fn(obs_data, {}))  # エージェントの反応を表示します

In [ ]:
# 手動で質問エージェントを実行します
@dataclass
class ObsDataIn(ObsData):
    turnType: str
    
obs_data = ObsDataIn(
        turn_type="",
        turnType="ask",  # ターンタイプを「質問」に設定します
        questions=["Is it a place?"],  # 質問のリスト
        answers=["Yes"],  # 既に得られた回答
        keyword="",  # キーワードはまだ指定されていないため空
        category=""  # カテゴリーも指定されていないため空
    )

# agent_fnを呼び出して反応を印刷します
print(agent_fn(obs_data, {}))  # エージェントの反応を表示します

In [ ]:
# 手動で推測エージェントを実行します
@dataclass
class ObsDataIn(ObsData):
    turnType: str
    
obs_data = ObsDataIn(
        turn_type="",
        turnType="guess",  # ターンタイプを「推測」に設定します
        questions=["Is it a place?", "Is it in the northern hemisphere?", "Is it a city?", "Is it icy?"],  # 質問のリスト
        answers=["Yes", "No", "No", "Yes"],  # 各質問に対する回答リスト
        keyword="",  # キーワードはまだ指定されていないため空
        category=""  # カテゴリーも指定されていないため空
    )

# agent_fnを呼び出して反応を印刷します
print(agent_fn(obs_data, {}))  # エージェントの反応を表示します